In [185]:
import re
from collections import defaultdict, Counter
from math import prod

from functools import cache

import black
import jupyter_black
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2023/{day}-example.txt" if example else f"2023/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [94]:
# Day 1: Trebuchet?!
numbers = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
}


def calibration_value(line, part1=True):
    numbers_found = ""
    for start, char in enumerate(line):
        if char.isdigit():
            numbers_found += char
        if part1:
            continue
        for key, value in numbers.items():
            if line.startswith(key, start):
                numbers_found += value
    calibration_value = numbers_found[0] + numbers_found[-1]
    return int(calibration_value)


games = data(1)
points = sum([calibration_value(line) for line in games])
print(f"Part 1: {points}")  # 53921
points = sum([calibration_value(line, part1=False) for line in games])
print(f"Part 2: {points}")  # 54676

Part 1: 53921
Part 2: 54676


In [151]:
# Day 2: Cube Conundrum
games = data(2)
possible_games = 0
powers = []
for game_nr, game in enumerate(games, 1):
    max_cubes_seen = {"red": 0, "blue": 0, "green": 0}
    cubes_seen = game.split(": ")[1].split("; ")
    for cubes in cubes_seen:
        for cube in cubes.split(", "):
            count, color = cube.split()
            count = int(count)
            if count > max_cubes_seen[color]:
                max_cubes_seen[color] = count
    if (
        max_cubes_seen["red"] <= 12
        and max_cubes_seen["green"] <= 13
        and max_cubes_seen["blue"] <= 14
    ):
        possible_games += game_nr
    powers.append(prod(max_cubes_seen.values()))

print(f"Part 1: {possible_games}")  # 2563
print(f"Part 2: {sum(powers)}")  # 70768

Part 1: 2563
Part 2: 70768


In [181]:
# Day 3: Gear Ratios
def neighbors(point):
    row, col = point
    potential = (
        (row - 1, col - 1),
        (row, col - 1),
        (row + 1, col - 1),
        (row - 1, col),
        (row + 1, col),
        (row - 1, col + 1),
        (row, col + 1),
        (row + 1, col + 1),
    )
    for (r, c) in potential:
        if 0 <= r < len(grid[0]) and 0 <= c < len(grid):
            yield (r, c)


grid = data(3)
gears = defaultdict(list)
part_numbers = []

for row_number, row in enumerate(grid):
    current_number = ""
    to_check = set()
    # Add '.' to catch numbers at end of line and avoid separate check after loop
    # completion
    for col_number, char in enumerate(row + "."):
        if char.isdigit():
            current_number += char
            for point in neighbors((row_number, col_number)):
                to_check.add(point)
        elif current_number:
            for (r, c) in to_check:
                if grid[r][c] not in "0123456789.":
                    # Symbol found, we are a part_number
                    part_numbers.append(int(current_number))
                if grid[r][c] == "*":
                    # Gear found, add number connected to this gear
                    gears[(r, c)].append(int(current_number))
            current_number = ""
            to_check = set()
print(f"Part 1: {sum(part_numbers)}")  # 514969
part2 = sum((prod(gear) for gear in gears.values() if len(gear) == 2))
print(f"Part 2: {part2}")  # 78915902

Part 1: 514969
Part 2: 78915902


In [94]:
# Day 4: Scratchcards
cards = data(4)
points = 0
total_cards = Counter([*range(len(cards))])  # One of each card to begin with
for card_nr, card in enumerate(cards):
    winning_numbers, my_numbers = map(ints, card.split(": ")[1].split(" | "))
    winners = sum(1 for number in my_numbers if number in winning_numbers)
    if winners:
        points += 2 ** (winners - 1)
        total_cards.update(
            total_cards[card_nr] * [*range(card_nr + 1, card_nr + winners + 1)]
        )

print(f"Part 1: {points}")  # 28750
print(f"Part 2: {sum(total_cards.values())}")  # 10212704

Part 1: 28750
Part 2: 10212704


In [160]:
# Day 5: If You Give A Seed A Fertilizer
def get_destination(seed, mapping):
    for dest, source, range_ in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        if source <= seed <= source + range_:
            return dest - source + seed
    return seed


def get_location(seed, mappings):
    for mapping in mappings:
        seed = get_destination(seed, mapping)
    return seed


chunks = data(5, sep="\n\n")
seeds = ints(chunks[0])
mappings = [tuple(ints(chunk)) for chunk in chunks[1:]]

print(f"Part 1: {min(get_location(seed, mappings) for seed in seeds)}")

Part 1: 462648396


In [184]:
chunks = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4""".split(
    "\n\n"
)

# Day 5: If You Give A Seed A Fertilizer
def get_destination(seed, mapping):
    for dest, source, range_ in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        if source <= seed <= source + range_:
            return dest - source + seed
    return seed


def get_location(seed, mappings):
    for mapping in mappings:
        seed = get_destination(seed, mapping)
    return seed


# chunks = data(5, sep="\n\n")

seeds = ints(chunks[0])
mappings = [ints(chunk) for chunk in chunks[1:]]
start = 1
for seed in range(start, start + 50):
    print(seed, get_location(seed, mappings))

1 23
2 24
3 25
4 26
5 27
6 28
7 29
8 30
9 31
10 32
11 33
12 34
13 35
14 43
15 36
16 37
17 38
18 39
19 40
20 41
21 42
22 43
23 91
24 92
25 93
26 1
27 2
28 3
29 4
30 5
31 6
32 7
33 8
34 9
35 10
36 11
37 12
38 13
39 14
40 15
41 16
42 17
43 18
44 61
45 62
46 63
47 64
48 65
49 66
50 20


In [216]:
# Day 6: Wait For It
times, distances = data(6, ints)

# Travel distance = speed * (time - button_press), where speed = button_press
part1 = prod(
    sum(
        1
        for button_press in range(time)
        if button_press * (time - button_press) > distance
    )
    for time, distance in zip(times, distances)
)
print(f"Part 1: {part1}")  # 512295

time = int("".join(map(str, times)))
distance = int("".join(map(str, distances)))
part2 = sum(
    1 for button_press in range(time) if button_press * (time - button_press) > distance
)
print(f"Part 2: {part2}")  # 36530883 (6s)

Part 1: 512295
Part 2: 36530883
